## extract the dictionary

In [ ]:
import json
import gzip

input_file = '/projects_bg/bg/shared/projects/gendas/gendas_benchmark/all_consequence_signature_per_gene.json.gz'
with gzip.open(input_file, 'rt') as f:
    d = json.load(f)

In [30]:
lex_keys = []
CB = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
subs = ['CA', 'CG', 'CT', 'TA', 'TC', 'TG', 
        'AC', 'AG', 'AT', 'GA', 'GC', 'GT']
for i in range(len(subs)):
    for b1 in sorted(CB):
        for b2 in sorted(CB):
            context = "".join([b1, b2])
            lex_keys.append(tuple([subs[i], context]))

## keep it in a numpy.array 

In [96]:
import numpy as np
gene_index = [item['GENE'] for item in d]
key_index = list(map(lambda x: x[1][0] + x[0][0] + x[1][1] + '>' + x[0][1], sorted(lex_keys)))
csqn_index = ['synonymous_variant', 'missense_variant', 'stop_gained_variant', 
                 'splice_acceptor_variant', 'splice_donor_variant']

In [97]:
csqn_index

['synonymous_variant',
 'missense_variant',
 'stop_gained_variant',
 'splice_acceptor_variant',
 'splice_donor_variant']

In [98]:
site_count_matrix = np.zeros((len(gene_index), len(key_index), len(csqn_index)))
for item in d:
    i = gene_index.index(item['GENE'])
    for key in item['context']:
        try:
            j = key_index.index(key)
        except:
            pass
        for csqn in item['context'][key]:
            if csqn in csqn_index:
                k = csqn_index.index(csqn)
                site_count_matrix[i, j, k] = item['context'][key][csqn]

In [99]:
import dill as pickle
import gzip

site_count = {'gene_index': gene_index, 
              'key_index': key_index, 
              'csqn_index': csqn_index, 
              'matrix': site_count_matrix}
with gzip.open('site_counts.pickle.gz', 'wb') as f:
    pickle.dump(site_count, f)

The array kept in 'matrix' has 3 indexes:

- 1st: gene index
- 2nd: mutational key index
- 3rd: consequence type index